<div style="text-align: right"> __Provided on May 14, Due on May 27__ [BRI516, Spring/2019] </div>

For homework in general:
* Install `Anaconda` and create an environment with `NumPy`, `Pandas`, `Matplotlib`, `scikit-learn` in Python 3.5 
* Please upload your jupyter-notebook file for homework to `Blackboard`
* Please visualize the results as much as possible and discuss your results at least one line of text

### [HW04]

#### Clustering Analysis (CH. 11)

##### (1) Data preparation
    Generate four isotropic Gaussian blobs with 400 2D samples and with cluster standard deviation of 0.5

##### (2) k-means
    (2a) Implement the k-means algorithm using generic python code (i.e., not using the scikit-learn)

    (2b) Implement the `k-means++` initialization method

    (2c) Run your k-means algorithm implementation using the blob dataset from (1) in the following conditions. Choose the reasonable number of runs and maximum iterations from a trial and error. 

    (2c-1) Choose an optimal number of clusters between 1 and 10 based on both the elbow method and silhouette method

    (2c-2) Set the optimal number of cluseters found from (2c-1) and compare the results of clustering analysis between the random initialization and `k-means++` initialization (implemented in 2b)  

    (2c-3) Compare the results of your implementation and results of the scikit-learn implementation for both (2c-1) and (2c-2)

##### (3) Fuzzy C-means (FCM)
    (3a) Implement the FCM using generic python code

    (3b) Let's suppose the fuzziness coefficient, m = 2. Choose the proper number of clusters between 1 and 10 based on the elbow method 

    (3c) Discuss the obtained results with the results from (2c-1)
